In [1]:
import pandas as pd
from hamcrest import assert_that, equal_to
import itertools

from er_evaluation.error_analysis import (
    record_error_table,
    cluster_sizes_from_table
)
from er_evaluation.estimators import (
    pairwise_recall_estimator,
    pairwise_precision_estimator,
)
from er_evaluation.estimators.from_table import (
    pairwise_precision_estimator_from_table,
    pairwise_recall_estimator_from_table,
)

def run_test_suite(*args):
    for prediction, sample in itertools.combinations_with_replacement(args, 2):
        error_table = record_error_table(prediction, sample)
        cs = cluster_sizes_from_table(error_table)
        weights = 1/cs

        assert_that(
            pairwise_precision_estimator(prediction, sample, weights="cluster_size"),
            equal_to(
                pairwise_precision_estimator_from_table(error_table, weights)
            )
        )
        assert_that(
            pairwise_recall_estimator(prediction, sample, weights="cluster_size"),
            equal_to(
                pairwise_recall_estimator_from_table(error_table, weights)
            )
        )

## Test two different precision/recall estimator computations are equal 

In [2]:
prediction = pd.Series(index=[1,2,3,4,5,6,7,8,9], data=[1,1,2,3,2,4,4,4,4])
reference = pd.Series(index=[1,2,3,4,5,6,7,8,9], data=["c1", "c1", "c1", "c2", "c2", "c3", "c3", "c4", "c4"])
sample = pd.Series(index=[1,2,3,4,5], data=["c1", "c1", "c1", "c2", "c2"])

run_test_suite(prediction, sample, reference)
run_test_suite(prediction, reference, reference)

### Check for RLdata

In [4]:
from er_evaluation.datasets import load_rldata500_disambiguations

predictions, reference = load_rldata500_disambiguations()
pred = list(predictions.values())[0]

run_test_suite(pred, reference)